In [16]:
### Getting first data source
# Source 2: a local file
# Top 100 songs with the most streams on Spotify
# Source --> https://www.kaggle.com/code/ludovicocuoghi/spotify-top-100-streamed-songs-analysis/input


import pandas as pd
print("Getting first data source...")
print("...reading...")
try:
    top_100_file_path = "/content/Top 100 most Streamed - Sheet1.csv"
    top_100_df = pd.read_csv(top_100_file_path)
    print("...first data source received...")
    print(f"...read in {top_100_df.shape[0]} rows (songs) across {top_100_df.shape[1]} columns")
except Exception as e:
    print("...failed to read given static source- ensure the file is in the content ('/content/<file>') directory in Google Colab")
    print(e)

Getting first data source...
...reading...
...first data source received...
...read in 100 rows (songs) across 14 columns


In [17]:
### Getting second data source
# Source 1: API data
# Using musicbrainz to match each artist/group in Source 2 with their country
# Using multiple API calls to make a new dataset

import requests
import time


try:
    artist_locations_df = pd.DataFrame(columns=["artist", "country"])
    index = 0

    url = "https://musicbrainz.org/ws/2/artist"
    artist_list = top_100_df["artist"]
    seen = set() # tracking queried artists to avoid API calls for speed

    print("Getting second data source...")
    print("...querying endpoints (takes around a minute because of intentional query delays)...")
    for artist in artist_list:
        if artist not in seen:
            # standard API call
            params = {
                "query": f'artist:"{artist}"',
                "fmt": "json"
            }
            headers = {
                "User-Agent": "ArtistLocations/2.0 ( fakeemail@example.com )"  # had to work around musicbrainz for query throttling
            }
            try:
                time.sleep(0.5)
                response = requests.get(url, headers=headers, params=params)
                artist_data = response.json()["artists"][0]

                # storing response in new df (building the data set)
                artist_locations_df.loc[index] = [artist, artist_data["country"]]
                index += 1
            except Exception as e:
                print("\t...musicbrainz could not gather info on " + artist + "...")
            seen.add(artist)
    print("...done gathering artist info, second dataset built...")
    print(f"...read in {artist_locations_df.shape[0]} rows (artists) across {artist_locations_df.shape[1]} columns")
except Exception as e:
    print('...failed to query MusicBrainz API, make sure previous code cell has been run')

Getting second data source...
...querying endpoints (takes around a minute because of intentional query delays)...
	...musicbrainz could not gather info on Harry Styles...
	...musicbrainz could not gather info on Calvin Harris...
	...musicbrainz could not gather info on Mark Mendy...
...done gathering artist info, second dataset built...
...read in 61 rows (artists) across 2 columns


In [19]:
### Merging + Analysis

print("Starting analysis of datasets...")
# merging and cleaning df
print("...merging and cleaning datasets...")
try:
    merged_df = top_100_df.merge(artist_locations_df, on="artist", how="inner")
    merged_df.drop(columns=["liveness", "year", "popularity", "acousticness", "speechiness", "loudness.dB"], inplace=True)
    print(f"...merged datasets into {merged_df.shape[0]} rows (songs + artist info) across {merged_df.shape[1]} columns\n\n")

    current_area_bpm_df = merged_df.groupby("country")["top genre"]
    print(current_area_bpm_df.describe())
    print("--------------\n")
    print("...grouping by country area and bpm...")
    current_area_bpm_df = merged_df.groupby("country")["beats.per.minute"]
    print(current_area_bpm_df.describe())
    print("--------------\n")
    print("...grouping by country area and energy...")
    current_area_bpm_df = merged_df.groupby("country")["energy"]
    print(current_area_bpm_df.describe())
    print("--------------\n")
    print("...grouping by country area and danceability...")
    current_area_bpm_df = merged_df.groupby("country")["danceability"]
    print(current_area_bpm_df.describe())
    print("--------------\n")
    print("...grouping by country area and valance...")
    current_area_bpm_df = merged_df.groupby("country")["valance"]
    print(current_area_bpm_df.describe())
    print("--------------\n")
    print("...grouping by country and length...")
    current_area_bpm_df = merged_df.groupby("country")["length"]
    print(current_area_bpm_df.describe())
    print("--------------\n")

    print("...analysis complete, formatted tables printed above and more specific conclusions found in the code comments")

except Exception as e:
    print("...analysis failed, make sure all previous code blocks have been run")
    print(e)



### Analysis Notes
# Note --> not a huge dataset, could effect analysis based on location if only a few songs per location (but that's not the focus of this project)
# Genre
    # Pop (or some variation of it) is by far the most popular amongst all countries
# BPM - beats-per-minute; the tempo of the song
    # US, GB, and Canada had high variability (std), as compared to Australia, Peurto Rico and Jamacia
    # Peurto Rico very high mean, Norway very low mean
# Energy - how energetic a song is (1-100)
    # Jamacia and Puertro Rico had low std's (outlier)
    # Puerto Rico, Sweden, and Jamacia had a very high mean
    # US, UK, Canada all had means on the lower end
# Danceability - how easy a song is to dance to (1-100)
    # Puertro Rico had low std (outlier)
    # high mean with Guyana, Jamacia, and Demark
    # Lower mean for Sweden and Norway
# Valance - the mood of the song, high = positive(1-100)
    # Puerto Rico very low std (outlier)
    # Puerto Rico also very high mean valance along side Guyana
    # Norway and France low mean
# Length - duration (seconds)
    # Low variation in Australia, Jamacia, and Peutro Rico
    # Guyana and Jamacia shorter mean
    # Great Britain and Sweden higher mean
# Overall
    # Pop (or some variation of it) is by far the most popular regardless of location
    # Puerto Rico stands out with high BPM, energy, and valance.
    # The US, Canada, and Great Britain have the most variation across all levels, suggesting more diverse types of songs (also maybe attributed to more top songs in the dataset).
    # There does seem to be some correlation with where an Artist is from and the attributes of their songs across the top 100 songs on Spotify
        # Somewhat hard to identify this trend across regions because only have country data, not country specifics (like weather, coordinates, etc.)

Starting analysis of datasets...
...merging and cleaning datasets...
...merged datasets into 97 rows (songs + artist info) across 9 columns


        count unique                      top freq
country                                           
AU          4      4           australian pop    1
CA         12      3             canadian pop    6
DK          1      1               danish pop    1
FR          1      1                dance pop    1
GB         15      5                      pop    7
GY          1      1              melodic rap    1
IE          1      1  irish singer-songwriter    1
JM          3      1                dance pop    3
NO          1      1            electro house    1
PR          2      1                    latin    2
SE          1      1                dance pop    1
US         55     20                dance pop   17
--------------

...grouping by country area and bpm...
         count        mean        std    min    25%    50%     75%    max
country        

In [20]:
### Generate SQL Database
import sqlite3
from pathlib import Path

print("Inserting data into a SQL database...")
try:
    # Creating empty file with sqlite database
    Path("data_project_1.db").touch()
    print("...empty database created...")

    # Connect to database
    conn = sqlite3.connect("data_project_1.db")
    merged_df.to_sql("TopSongsWithArea", conn, if_exists="replace", index=False)
    print("...database filled...")
    conn.close()
    print("...finished, connection closed")
except Exception as e:
    print("...something went wrong when generating the SQL database, make sure all previous code blocks have been run")
    print(e)


Inserting data into a SQL database...
...empty database created...
...database filled...
...finished, connection closed


In [24]:
### Return type to the user

print("How would you like to return merged datasets? 'C' for a .csv file, 'J' for a .json file, or 'S' for SQL")
try:
    return_type = str(input()).lower()
    if return_type == "c":
        print("...csv selected, generating file...")
        merged_df.to_csv("TopSongsWithArea.csv", index=False)
        print("...csv generated as TopSongsWithArea.csv in the /content/ directory")
    elif return_type == "j":
        print("...json selected, generating file...")
        merged_df.to_json("TopSongsWithArea.json", orient="records")
        print("...json generated as TopSongsWithArea.json in the /content/ directory")
    elif return_type == "s":
        print("...SQL selected, generating file...")
        print("...connecting to the database...")
        try:
            conn = sqlite3.connect("data_project_1.db")
            print("...connected, reading data...")
            with open("TopSongsWithArea.sql", "w") as file:
                for line in conn.iterdump():
                    file.write(f"{line}\n")
            print("...SQL generated as TopSongsWithArea.sql in the /content/ directory")
        except Exception as e:
            print("...error occured while connecting to the database or writing the .sql file. Make sure all previous code blocks have been run")
    else:
        print("...please enter a valid input")
except Exception as e:
    print("...something went wrong with your return input, please ensure it is a valid matching string and that all previous code blocks have been run")
    print(e)

How would you like to return merged datasets? 'C' for a .csv file, 'J' for a .json file, or 'S' for SQL
s
...SQL selected, generating file...
...connecting to the database...
...connected, reading data...
...SQL generated as TopSongsWithArea.sql in the /content/ directory
